In [2]:
import yfinance as yf
import pandas as pd

# Step 1: Fetch USD/CAD exchange rate data using yfinance
def get_usd_cad_data_yf():
    # Fetching the USD/CAD exchange rate data using ticker symbol "CAD=X"
    usd_cad = yf.download("CAD=X", start="2015-01-01", end="2024-01-01")
    # Adjusting the dataframe to keep only the adjusted close prices
    usd_cad = usd_cad[['Adj Close']]
    usd_cad.rename(columns={'Adj Close': 'usd_cad'}, inplace=True)
    return usd_cad

# Fetch the data
usd_cad_data = get_usd_cad_data_yf()

# Display the first few rows
usd_cad_data_yf.head()


[*********************100%***********************]  1 of 1 completed
C:\Users\nilee\AppData\Local\Temp\ipykernel_6428\3947655444.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usd_cad.rename(columns={'Adj Close': 'usd_cad'}, inplace=True)


,usd_cad
Date,
2015-01-01,1.16146
2015-01-02,1.16140
2015-01-05,1.17930
2015-01-06,1.17570
2015-01-07,1.18370


In [4]:
import numpy as np

# Step 2: Feature Engineering

# Calculate technical indicators
usd_cad_data['SMA20'] = usd_cad_data['usd_cad'].rolling(window=20).mean()
usd_cad_data['SMA50'] = usd_cad_data['usd_cad'].rolling(window=50).mean()

# Calculate RSI
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

usd_cad_data['RSI'] = calculate_rsi(usd_cad_data['usd_cad'])

# Calculate momentum
usd_cad_data['Momentum'] = usd_cad_data['usd_cad'].diff(4)

# Create trend feature: if the short-term moving average is above the long-term moving average, it's an uptrend
usd_cad_data['Trend'] = np.where(usd_cad_data['SMA20'] > usd_cad_data['SMA50'], 1, 0)

# Drop missing values (due to rolling windows)
usd_cad_data.dropna(inplace=True)

# Create target variable: Shift USD/CAD prices by one day to predict future price movement
usd_cad_data['Target'] = usd_cad_data['usd_cad'].shift(-1)

# Remove rows with missing target values (last row will have NaN for Target due to shifting)
usd_cad_data.dropna(inplace=True)

# Display the updated dataframe
usd_cad_data.head()


,usd_cad,SMA20,SMA50,RSI,Momentum,Trend,Target
Date,,,,,,,
2015-05-19,1.21450,1.209016,1.236906,57.631445,0.01420,0,1.22291
2015-05-20,1.22291,1.208829,1.236015,63.626944,0.02751,0,1.22064
2015-05-21,1.22064,1.208650,1.234950,58.720374,0.02234,0,1.21960
2015-05-22,1.21960,1.208896,1.233960,52.565618,0.01842,0,1.22950
2015-05-25,1.22950,1.209524,1.232917,64.629387,0.01500,0,1.23193
